In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
Epoch 1/150
196/196 [==============================] - 8s 42ms/step - loss: 2.3175 - accuracy: 0.3348 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 4s 23ms/step - loss: 1.7075 - accuracy: 0.4669 - val_loss: 1.8623 - val_accuracy: 0.4163
Epoch 3/150
196/196 [==============================] - 4s 23ms/step - loss: 1.5794 - accuracy: 0.5236 - val_loss: 1.3704 - val_accuracy: 0.5759
Epoch 4/150
196/196 [==============================] - 4s 23ms/step - loss: 1.3880 - accuracy: 0.5800 - val_loss: 1.2979 - val_accuracy: 0.6138
Epoch 5/150
196/196 [==============================] - 4s 23ms/step - loss: 1.2355 - accuracy: 0.6332 - val_loss: 1.0643 - val_accuracy: 0.6868
Epoch 6/150
196/196 [==============================] - 4s 22ms/step - loss: 1.1269 - accuracy: 0.6689 - val_loss: 1.0032 - val_accuracy: 0.7087
Epoch 7/150
196/196 [==============================] - 4s 22ms/ste

196/196 [==============================] - 4s 23ms/step - loss: 0.6152 - accuracy: 0.8776 - val_loss: 0.6221 - val_accuracy: 0.8790
Epoch 58/150
196/196 [==============================] - 4s 23ms/step - loss: 0.6141 - accuracy: 0.8793 - val_loss: 0.6230 - val_accuracy: 0.8808
Epoch 59/150
196/196 [==============================] - 4s 23ms/step - loss: 0.6108 - accuracy: 0.8792 - val_loss: 0.6221 - val_accuracy: 0.8856
Epoch 60/150
196/196 [==============================] - 4s 23ms/step - loss: 0.6111 - accuracy: 0.8805 - val_loss: 0.6271 - val_accuracy: 0.8806
Epoch 61/150
196/196 [==============================] - 4s 23ms/step - loss: 0.6061 - accuracy: 0.8818 - val_loss: 0.6093 - val_accuracy: 0.8816
Epoch 62/150
196/196 [==============================] - 4s 23ms/step - loss: 0.6057 - accuracy: 0.8808 - val_loss: 0.6374 - val_accuracy: 0.8798
Epoch 63/150
196/196 [==============================] - 4s 23ms/step - loss: 0.6050 - accuracy: 0.8832 - val_loss: 0.6273 - val_accuracy: 0.882

In [4]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### remove data augmentation ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
Epoch 1/150
196/196 [==============================] - 6s 32ms/step - loss: 2.1497 - accuracy: 0.3679 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 4s 22ms/step - loss: 1.5780 - accuracy: 0.5152 - val_loss: 1.8076 - val_accuracy: 0.4358
Epoch 3/150
196/196 [==============================] - 4s 22ms/step - loss: 1.3180 - accuracy: 0.6012 - val_loss: 1.2360 - val_accuracy: 0.6306
Epoch 4/150
196/196 [==============================] - 4s 22ms/step - loss: 1.1304 - accuracy: 0.6631 - val_loss: 1.0247 - val_accuracy: 0.7028
Epoch 5/150
196/196 [==============================] - 4s 22ms/step - loss: 1.0337 - accuracy: 0.7003 - val_loss: 0.9484 - val_accuracy: 0.7338
Epoch 6/150
196/196 [==============================] - 4s 22ms/step - loss: 0.9414 - accuracy: 0.7314 - val_loss: 0.9173 - val_accuracy: 0.7472
Epoch 7/150
196/196 [==============================] - 4s 22ms/ste

196/196 [==============================] - 4s 23ms/step - loss: 0.4926 - accuracy: 0.9336 - val_loss: 0.7813 - val_accuracy: 0.8656
Epoch 58/150
196/196 [==============================] - 4s 22ms/step - loss: 0.4920 - accuracy: 0.9343 - val_loss: 0.7866 - val_accuracy: 0.8670
Epoch 59/150
196/196 [==============================] - 4s 22ms/step - loss: 0.4910 - accuracy: 0.9336 - val_loss: 0.7837 - val_accuracy: 0.8642
Epoch 60/150
196/196 [==============================] - 4s 22ms/step - loss: 0.4889 - accuracy: 0.9351 - val_loss: 0.7824 - val_accuracy: 0.8683
Epoch 61/150
196/196 [==============================] - 4s 23ms/step - loss: 0.4874 - accuracy: 0.9362 - val_loss: 0.8075 - val_accuracy: 0.8630
Epoch 62/150
196/196 [==============================] - 4s 23ms/step - loss: 0.4865 - accuracy: 0.9369 - val_loss: 0.7896 - val_accuracy: 0.8676
Epoch 63/150
196/196 [==============================] - 4s 22ms/step - loss: 0.4921 - accuracy: 0.9355 - val_loss: 0.7757 - val_accuracy: 0.869

In [7]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### remove data augmentation and  kernel_regularizer and Dropout ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
Epoch 1/150
196/196 [==============================] - 6s 29ms/step - loss: 1.4911 - accuracy: 0.5073 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 4s 22ms/step - loss: 0.9542 - accuracy: 0.6681 - val_loss: 1.2703 - val_accuracy: 0.5651
Epoch 3/150
196/196 [==============================] - 4s 21ms/step - loss: 0.7144 - accuracy: 0.7508 - val_loss: 0.8798 - val_accuracy: 0.6984
Epoch 4/150
196/196 [==============================] - 4s 21ms/step - loss: 0.5665 - accuracy: 0.8004 - val_loss: 0.8482 - val_accuracy: 0.7304
Epoch 5/150
196/196 [==============================] - 4s 21ms/step - loss: 0.4512 - accuracy: 0.8412 - val_loss: 0.8414 - val_accuracy: 0.7370
Epoch 6/150
196/196 [==============================] - 4s 22ms/step - loss: 0.3585 - accuracy: 0.8740 - val_loss: 1.0429 - val_accuracy: 0.7218
Epoch 7/150
196/196 [==============================] - 4s 21ms/ste

196/196 [==============================] - 4s 21ms/step - loss: 0.0172 - accuracy: 0.9939 - val_loss: 1.4496 - val_accuracy: 0.7971
Epoch 58/150
196/196 [==============================] - 4s 21ms/step - loss: 0.0232 - accuracy: 0.9925 - val_loss: 1.5390 - val_accuracy: 0.7982
Epoch 59/150
196/196 [==============================] - 4s 21ms/step - loss: 0.0268 - accuracy: 0.9910 - val_loss: 1.5344 - val_accuracy: 0.7886
Epoch 60/150
196/196 [==============================] - 4s 22ms/step - loss: 0.0296 - accuracy: 0.9901 - val_loss: 1.4888 - val_accuracy: 0.7903
Epoch 61/150
196/196 [==============================] - 4s 21ms/step - loss: 0.0337 - accuracy: 0.9884 - val_loss: 1.4170 - val_accuracy: 0.8030
Epoch 62/150
196/196 [==============================] - 4s 21ms/step - loss: 0.0202 - accuracy: 0.9934 - val_loss: 1.3614 - val_accuracy: 0.8026
Epoch 63/150
196/196 [==============================] - 4s 21ms/step - loss: 0.0126 - accuracy: 0.9960 - val_loss: 1.4345 - val_accuracy: 0.804

In [10]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### remove data augmentation and  kernel_regularizer and Dropout and BatchNormalization ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
Epoch 1/150
196/196 [==============================] - 5s 23ms/step - loss: 1.6415 - accuracy: 0.3935 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 3s 17ms/step - loss: 1.1143 - accuracy: 0.5967 - val_loss: 1.0839 - val_accuracy: 0.6122
Epoch 3/150
196/196 [==============================] - 3s 17ms/step - loss: 0.8656 - accuracy: 0.6913 - val_loss: 0.8469 - val_accuracy: 0.7016
Epoch 4/150
196/196 [==============================] - 3s 17ms/step - loss: 0.7065 - accuracy: 0.7518 - val_loss: 0.7605 - val_accuracy: 0.7388
Epoch 5/150
196/196 [==============================] - 3s 18ms/step - loss: 0.5740 - accuracy: 0.7990 - val_loss: 0.7482 - val_accuracy: 0.7441
Epoch 6/150
196/196 [==============================] - 3s 18ms/step - loss: 0.4727 - accuracy: 0.8351 - val_loss: 0.7199 - val_accuracy: 0.7640
Epoch 7/150
196/196 [==============================] - 3s 17ms/ste

196/196 [==============================] - 4s 18ms/step - loss: 0.0446 - accuracy: 0.9862 - val_loss: 1.7003 - val_accuracy: 0.7770
Epoch 58/150
196/196 [==============================] - 4s 18ms/step - loss: 0.0419 - accuracy: 0.9863 - val_loss: 1.7441 - val_accuracy: 0.7697
Epoch 59/150
196/196 [==============================] - 4s 18ms/step - loss: 0.0419 - accuracy: 0.9869 - val_loss: 1.7845 - val_accuracy: 0.7738
Epoch 60/150
196/196 [==============================] - 4s 18ms/step - loss: 0.0414 - accuracy: 0.9868 - val_loss: 1.6175 - val_accuracy: 0.7720
Epoch 61/150
196/196 [==============================] - 4s 18ms/step - loss: 0.0359 - accuracy: 0.9885 - val_loss: 1.7191 - val_accuracy: 0.7738
Epoch 62/150
196/196 [==============================] - 3s 18ms/step - loss: 0.0427 - accuracy: 0.9861 - val_loss: 1.8016 - val_accuracy: 0.7736
Epoch 63/150
196/196 [==============================] - 4s 18ms/step - loss: 0.0482 - accuracy: 0.9839 - val_loss: 1.8545 - val_accuracy: 0.771

In [14]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code - use default l1_l2 ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 0.01

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
Epoch 1/150
196/196 [==============================] - 6s 32ms/step - loss: 47.9913 - accuracy: 0.2845 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 4s 22ms/step - loss: 5.4571 - accuracy: 0.3237 - val_loss: 4.5819 - val_accuracy: 0.1000
Epoch 3/150
196/196 [==============================] - 4s 23ms/step - loss: 3.4572 - accuracy: 0.3889 - val_loss: 3.6177 - val_accuracy: 0.2297
Epoch 4/150
196/196 [==============================] - 4s 22ms/step - loss: 2.9876 - accuracy: 0.4343 - val_loss: 2.9528 - val_accuracy: 0.4100
Epoch 5/150
196/196 [==============================] - 4s 22ms/step - loss: 2.7418 - accuracy: 0.4608 - val_loss: 2.7106 - val_accuracy: 0.4356
Epoch 6/150
196/196 [==============================] - 4s 22ms/step - loss: 2.5887 - accuracy: 0.4746 - val_loss: 2.5023 - val_accuracy: 0.4832
Epoch 7/150
196/196 [==============================] - 4s 23ms/st

196/196 [==============================] - 4s 22ms/step - loss: 2.3478 - accuracy: 0.5482 - val_loss: 2.2063 - val_accuracy: 0.6015
Epoch 58/150
196/196 [==============================] - 4s 22ms/step - loss: 2.3550 - accuracy: 0.5497 - val_loss: 2.4243 - val_accuracy: 0.5294
Epoch 59/150
196/196 [==============================] - 4s 23ms/step - loss: 2.3517 - accuracy: 0.5487 - val_loss: 2.3389 - val_accuracy: 0.5764
Epoch 60/150
196/196 [==============================] - 4s 23ms/step - loss: 2.3349 - accuracy: 0.5533 - val_loss: 2.2662 - val_accuracy: 0.5772
Epoch 61/150
196/196 [==============================] - 4s 22ms/step - loss: 2.3390 - accuracy: 0.5532 - val_loss: 2.3265 - val_accuracy: 0.5515
Epoch 62/150
196/196 [==============================] - 4s 22ms/step - loss: 2.3451 - accuracy: 0.5486 - val_loss: 2.2299 - val_accuracy: 0.5911
Epoch 63/150
196/196 [==============================] - 4s 22ms/step - loss: 2.3330 - accuracy: 0.5528 - val_loss: 2.3551 - val_accuracy: 0.551

In [17]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code-increase filters ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
Epoch 1/150
196/196 [==============================] - 13s 68ms/step - loss: 3.5409 - accuracy: 0.3352 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 9s 47ms/step - loss: 2.7865 - accuracy: 0.4102 - val_loss: 2.4317 - val_accuracy: 0.3920
Epoch 3/150
196/196 [==============================] - 9s 48ms/step - loss: 2.3354 - accuracy: 0.4961 - val_loss: 2.0758 - val_accuracy: 0.5314
Epoch 4/150
196/196 [==============================] - 9s 48ms/step - loss: 2.1576 - accuracy: 0.5516 - val_loss: 1.8848 - val_accuracy: 0.5808
Epoch 5/150
196/196 [==============================] - 9s 47ms/step - loss: 1.9640 - accuracy: 0.5918 - val_loss: 1.7602 - val_accuracy: 0.6377
Epoch 6/150
196/196 [==============================] - 9s 48ms/step - loss: 1.7801 - accuracy: 0.6375 - val_loss: 1.5946 - val_accuracy: 0.6843
Epoch 7/150
196/196 [==============================] - 9s 47ms/st

196/196 [==============================] - 9s 48ms/step - loss: 0.9488 - accuracy: 0.9043 - val_loss: 1.0883 - val_accuracy: 0.8789
Epoch 58/150
196/196 [==============================] - 9s 48ms/step - loss: 0.9515 - accuracy: 0.9068 - val_loss: 1.0044 - val_accuracy: 0.8937
Epoch 59/150
196/196 [==============================] - 9s 48ms/step - loss: 0.9440 - accuracy: 0.9066 - val_loss: 1.0360 - val_accuracy: 0.8844
Epoch 60/150
196/196 [==============================] - 9s 48ms/step - loss: 0.9243 - accuracy: 0.9098 - val_loss: 1.0061 - val_accuracy: 0.8917
Epoch 61/150
196/196 [==============================] - 9s 48ms/step - loss: 0.9149 - accuracy: 0.9108 - val_loss: 0.9967 - val_accuracy: 0.8900
Epoch 62/150
196/196 [==============================] - 9s 48ms/step - loss: 0.9331 - accuracy: 0.9075 - val_loss: 1.0295 - val_accuracy: 0.8907
Epoch 63/150
196/196 [==============================] - 9s 48ms/step - loss: 0.9361 - accuracy: 0.9101 - val_loss: 0.9937 - val_accuracy: 0.894

In [21]:
# add skip connection
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code - add skip connection and increase filters ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

# model
inputs = tf.keras.Input(shape=(32, 32, 3))
x = inputs

# block 1
previous = x

x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 2
previous = x

x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 3
previous = x

x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# block 4
previous = x

x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# dense block 

x = Flatten()(x)
x = Dense(128)(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs)

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
Epoch 1/150
196/196 [==============================] - 29s 148ms/step - loss: 4.1725 - accuracy: 0.3173 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 23s 117ms/step - loss: 2.8174 - accuracy: 0.4638 - val_loss: 2.6362 - val_accuracy: 0.5314
Epoch 3/150
196/196 [==============================] - 23s 117ms/step - loss: 2.4796 - accuracy: 0.5613 - val_loss: 2.4963 - val_accuracy: 0.5807
Epoch 4/150
196/196 [==============================] - 23s 117ms/step - loss: 2.2701 - accuracy: 0.6210 - val_loss: 2.4144 - val_accuracy: 0.5792
Epoch 5/150
196/196 [==============================] - 23s 117ms/step - loss: 2.0782 - accuracy: 0.6737 - val_loss: 2.0730 - val_accuracy: 0.6940
Epoch 6/150
196/196 [==============================] - 23s 117ms/step - loss: 1.9287 - accuracy: 0.7143 - val_loss: 1.7804 - val_accuracy: 0.7557
Epoch 7/150
196/196 [==============================] -

Epoch 112/150
196/196 [==============================] - 23s 117ms/step - loss: 0.5366 - accuracy: 0.9777 - val_loss: 0.7377 - val_accuracy: 0.9310
Epoch 113/150
196/196 [==============================] - 23s 117ms/step - loss: 0.5236 - accuracy: 0.9796 - val_loss: 0.7286 - val_accuracy: 0.9307
Epoch 114/150
196/196 [==============================] - 23s 117ms/step - loss: 0.5127 - accuracy: 0.9805 - val_loss: 0.7223 - val_accuracy: 0.9314
Epoch 115/150
196/196 [==============================] - 23s 117ms/step - loss: 0.5059 - accuracy: 0.9808 - val_loss: 0.7211 - val_accuracy: 0.9313
Epoch 116/150
196/196 [==============================] - 23s 117ms/step - loss: 0.4970 - accuracy: 0.9816 - val_loss: 0.7057 - val_accuracy: 0.9331
Epoch 117/150
196/196 [==============================] - 23s 117ms/step - loss: 0.4873 - accuracy: 0.9831 - val_loss: 0.7111 - val_accuracy: 0.9323
Epoch 118/150
196/196 [==============================] - 23s 117ms/step - loss: 0.4839 - accuracy: 0.9834 - val_

In [ ]:
# add skip connection
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code - add skip connection and increase filters ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

# model
inputs = tf.keras.Input(shape=(32, 32, 3))
x = inputs

# block 1
previous = x

x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 2
previous = x

x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.3)(x)


# block 3
previous = x

x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# block 4
previous = x

x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3,3), padding='same', activation='relu', 
           kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay))(x)
x = tfa.layers.InstanceNormalization()(x)
x = BatchNormalization()(x)

#x = keras.layers.concatenate([x, previous]) 

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


# dense block 

x = Flatten()(x)
x = Dense(128)(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs)

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

In [22]:
import numpy as np
accs = np.array(history.history['accuracy'])*100
for i in range(150):
    print(accs[i])

31.734
46.378
56.132
62.1
67.368004
71.43
73.944
76.426
77.728
79.416
80.232
81.068
82.294
82.770004
83.088
83.736
84.124
84.618004
85.068
85.144005
85.488
85.586
86.05801
86.21
86.548004
86.842
86.92
87.386
87.732
87.488
88.076004
88.224
88.254
88.404
88.394
88.768
89.018
89.020004
89.322
89.71
89.97
89.878
90.0
90.168
90.086
90.388
90.554
90.526
90.794
90.652
91.274
91.102
91.158005
91.266
91.364
91.439995
91.438
91.49
91.688
91.77
92.188
91.956
91.922
92.134
92.072
92.17
92.358
92.203995
92.15
92.813995
94.630005
94.85
95.108
95.136
95.406
95.146
95.341995
95.271996
95.437996
95.502
95.520004
95.57
95.46
95.572
95.386
95.514
95.466
95.71
95.586
95.502
95.53
95.740005
95.932
95.774
95.706
95.661995
95.854004
95.841995
95.742
95.734
95.856
95.75
95.98
95.895996
96.065994
95.894005
95.911995
95.788
95.984
96.468
97.368004
97.774
97.96
98.049995
98.079994
98.159996
98.30801
98.342
98.296
98.306
98.426
98.39
98.39
98.438
98.372
98.624
98.504
98.518005
98.622
98.59
98.556
98.689995
98.516

In [23]:
val_as = np.array(history.history['val_accuracy'])*100
for i in range(150):
    print(val_as[i])

38.72
53.140003
58.07
57.920002
69.4
75.57
75.63
77.57
78.829994
80.729996
78.869995
82.36
82.73
83.3
83.53001
83.37
82.78
82.31
82.6
84.49
84.28
85.46
83.83
85.17
85.439995
85.21999
85.88
85.299995
85.38
85.509995
86.729996
86.07
85.14
86.68
86.94
87.81
88.0
87.64
87.15
88.83
86.619995
87.52
88.41
88.66
86.39
88.33
89.04
88.85
88.08
88.990005
88.6
89.16
88.66
89.840004
87.98
89.490005
88.880005
88.42
88.08
89.21
89.170006
89.35
89.0
88.69
89.57
89.5
89.96001
89.5
89.64
91.61
91.78
91.7
91.909996
91.86
91.77
90.82
91.72
91.259995
91.509995
91.689995
91.46999
91.799995
91.85
90.7
91.42
91.78
91.409996
91.39
91.509995
91.329994
91.24
91.84
91.45
91.45
91.45
91.36
91.579994
91.509995
91.43
91.67
91.32
91.34
91.96
91.799995
91.85
91.77
91.729996
91.99
91.43
92.75
92.85
93.1
93.07
93.14
93.13
93.31
93.229996
93.02
93.15
93.22
93.09
92.95
93.07
92.909996
93.06
93.3
93.21
93.28
93.19
93.08
93.29
93.03
93.11
93.26
93.26
93.229996
93.159996
93.25
93.01
93.09
93.03
93.13
93.28
93.06
93.02
93.14


In [ ]:
Conv2D(num_filters, 
       kernel_size, 
       padding='same', 
       activation='relu',
       kernel_regularizer=regularizers.l1_l2(decay1, decay2))
